In [ ]:
import math
import torchinfo
import os
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
import copy



from torch import nn
from System.pos_enc import *
from torch.optim.lr_scheduler import StepLR
from Data.MowingDetectionDataset_Parcelized import *
from System.pos_enc import *
from System.Engine import *
#from System.Engine_Tr import *
from helpers.various import *
from torchmetrics import Accuracy, ConfusionMatrix, F1Score, Recall, Precision
from torchinfo import summary


transforms = A.Compose([
        #A.Normalize(mean=[-0.3357, -0.3259],#####how many channels to include?
                   # std=[0.4688, 0.4685]),
        ToTensorV2()
    ])




###padding is used because there is the possibility to have nan at the ts start###
###also the argument nan_free is passed to interpolate the ts###
train_dt = ParcelMowingDataset(root_path = root_dataset_path,labels_path = path_train, trsm = transforms,max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
train_dloader = DataLoader(train_dt, sampler=ImbalancedDatasetSampler(train_dt), 
                           batch_size=1, shuffle=False, num_workers=0)

val_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_val, trsm = transforms, max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
val_dloader = DataLoader(val_dt, batch_size=1, shuffle=False, num_workers=0)

test_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_test, trsm = transforms, max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
test_dloader = DataLoader(test_dt, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
dt, lbl = next(iter(test_dloader))
print(dt.shape)
print(lbl)

In [ ]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
        
    def __call__(self, module, module_in, module_out):
        self.outputs.append(module_out)
        
    def clear(self):
        self.outputs = []
        
        
        
import torch.nn.functional as F
import matplotlib.pyplot as plt
def module_output_to_numpy(tensor):
    # Apply softmax to dimension 1
    tensor_softmax = F.softmax(tensor, dim=1)
    # Convert tensor to numpy array
    return tensor_softmax.squeeze().detach().to('cpu').numpy()


In [ ]:
model = resnet()
save_output = SaveOutput()
hook_handles = []
for layer in model.modules():
    #print(layer)
    if isinstance(layer, torch.nn.Conv1d):
        handle = layer.register_forward_hook(save_output)
        hook_handles.append(handle)
        print('ok')
        
len(save_output.outputs)
model(dt)

len(save_output.outputs)
for out in save_output.outputs:
    print(out.squeeze().shape)

In [ ]:
best_model = ECGTRMODEL.load_from_checkpoint('./checkpoints/finetuned_models_hooks/lightning_logs/version_8/checkpoints/epoch=91-step=2882.ckpt')
trainer = pl.Trainer(accelerator="cpu", 
    devices=1)
trainer.test(best_model, dataloaders=test_dloader)

In [ ]:
#####to do!!!!####
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt


class_labels = ['0 times', '1 times','2 times', '3 times', '4 times']
cm = best_model.test_confusion_matrix.compute().cpu().numpy()

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=class_labels )
plt.figure(figsize=(5,5), dpi=100)
ax = plt.axes()

disp.plot(ax=ax)
plt.xticks(rotation=90)
plt.show()

In [ ]:
data_list = []
for dh in test_dloader:
    data_list.append(dh)
print(len(data_list))

In [ ]:
del save_output
save_output = SaveOutput()

hook_handles = []

for layer in best_model.modules():
    print(layer)
    if isinstance(layer, torch.nn.Conv1d):
        handle = layer.register_forward_hook(save_output)
        hook_handles.append(handle)
        print('placed hook in layer', layer)

len(save_output.outputs)
dt, lbl = data_list[31]
print(dt.shape)
print(lbl)
with torch.no_grad():
    y = best_model(dt)
print('Pred',torch.argmax(y), 'GT',lbl)

In [ ]:
len(save_output.outputs)
for i in save_output.outputs:
    print(i)
    break

In [ ]:
for out in save_output.outputs:
    print(out.squeeze().shape)

In [ ]:
tensor_2 = module_output_to_numpy(save_output.outputs[0])
tensor_4 = module_output_to_numpy(save_output.outputs[1])

tensor_4.shape
t2_matmult_4 = np.inner(tensor_2,tensor_4)
plt.imshow(t2_matmult_4.T)
print(t2_matmult_4.shape)

In [ ]:
tensor_x = module_output_to_numpy(save_output.outputs[3])
plt.imshow(tensor_x.T)

In [ ]:
tensor_5 = module_output_to_numpy(save_output.outputs[3])
tensor_6 = module_output_to_numpy(save_output.outputs[4])
t5_matmult_6 = np.inner(tensor_5,tensor_6)
plt.imshow(t5_matmult_6)

In [ ]:
for item in data_list:
    #print(item[1])

    with torch.no_grad():
        y = best_model(item[0])
        print(y)
    print('Pred',torch.argmax(y), 'GT',item[1])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Define the tensor
tensor = dt
# Convert tensor to numpy array
tensor_np = tensor.numpy()

# Filter out NaN values
tensor_np[np.isnan(tensor_np)] = 0

# Plot only non-NaN values
plt.figure(figsize=(8, 6))
plt.plot(tensor_np.squeeze())
plt.title("Time Serie Sample")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid(True)
plt.show()
